# LangGraph Tutorial: Single Tool Execution

## Objective
Understand the fundamental execution pattern when an agent uses a single tool to answer a query.

## What You'll Learn
1. The complete execution flow: START → Agent → Tools → Agent → END
2. How messages accumulate in state during execution
3. The two LLM calls pattern (decide + respond)
4. Message types and their roles (HumanMessage, AIMessage, ToolMessage)
5. How the router determines flow based on `tool_calls`

## Prerequisites
- Completed: Notebook 05 (Graph Construction)
- Understanding of the agent-tools-loop architecture

---

## Section 1: The Single Tool Pattern

### Execution Flow

```
User Query → Agent analyzes → Calls ONE tool → Tool executes → Agent responds
```

### Reference Point: What Happens Step by Step

| Step | Node | Action | Output |
|------|------|--------|--------|
| 1 | START | Entry point | Route to agent |
| 2 | agent | LLM decides to call tool | AIMessage with `tool_calls` |
| 3 | router | Sees `tool_calls` | Route to tools |
| 4 | tools | Executes tool function | ToolMessage with result |
| 5 | agent | LLM formulates answer | AIMessage with `content` |
| 6 | router | No `tool_calls` | Route to END |

### Message Count Pattern

```
Single tool execution always produces 4 messages:
  1. HumanMessage     (user query)
  2. AIMessage        (tool call request)
  3. ToolMessage      (tool result)
  4. AIMessage        (final response)
```

---

## Section 2: Setup

Rebuild the graph from Notebook 05.

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage
        tenure_months: Loan tenure in months
        currency: Currency code for display
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

In [ ]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

In [ ]:
# Build graph
def call_llm(state: MessagesState):
    """Agent node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Assemble the graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

---

## Section 3: Execute a Single Tool Query

**Query:** "What is 1000 USD in EUR?"

This query requires exactly ONE tool call (currency_converter).

In [ ]:
# Create initial state with user query
state = {
    "messages": [HumanMessage(content="What is 1000 USD in EUR?")]
}

print("Initial State:")
print("=" * 70)
print(f"Query: {state['messages'][0].content}")
print(f"Message count: {len(state['messages'])}")
print("=" * 70)

In [ ]:
# Execute the graph
result = app.invoke(state)

print("Execution Complete!")
print("=" * 70)
print(f"Final message count: {len(result['messages'])}")
print("=" * 70)

---

## Section 4: Examine the Message Flow

Let's inspect each message to understand what happened at each step.

In [ ]:
print("MESSAGE FLOW ANALYSIS")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 70}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 70}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"  🤖 AGENT DECISION: Call tool(s)")
            print(f"  Tool Calls: {len(msg.tool_calls)}")
            for tc in msg.tool_calls:
                print(f"    • Tool: {tc['name']}")
                print(f"      Args: {tc['args']}")
                print(f"      ID: {tc['id']}")
        else:
            print(f"  🤖 AGENT RESPONSE: Final answer")
            print(f"  Content: {msg.content}")
            
    elif isinstance(msg, ToolMessage):
        print(f"  🔧 TOOL RESULT")
        print(f"  Tool Call ID: {msg.tool_call_id}")
        print(f"  Content:\n{msg.content}")

### Reference Point: Message Anatomy

| Message Type | Key Attributes | Created By |
|--------------|----------------|------------|
| `HumanMessage` | `.content` | User input |
| `AIMessage` (tool call) | `.tool_calls` (list of dicts) | LLM deciding to use tool |
| `ToolMessage` | `.content`, `.tool_call_id` | ToolNode executing tool |
| `AIMessage` (response) | `.content` | LLM providing final answer |

### The `tool_calls` Structure

```python
msg.tool_calls = [
    {
        "name": "currency_converter",      # Which tool to call
        "args": {                          # Arguments extracted from query
            "amount": 1000,
            "from_currency": "USD",
            "to_currency": "EUR"
        },
        "id": "call_abc123"                # Unique ID for matching response
    }
]
```

---

## Section 5: Deep Dive into Each Message

In [ ]:
# Message 1: HumanMessage (User Query)
print("MESSAGE 1: HumanMessage")
print("=" * 70)
print(f"Type: {type(result['messages'][0]).__name__}")
print(f"Content: {result['messages'][0].content}")
print("\n→ This is the starting point. User's natural language query.")

In [ ]:
# Message 2: AIMessage with tool_calls (Agent's Decision)
print("MESSAGE 2: AIMessage (Tool Call Request)")
print("=" * 70)
msg2 = result['messages'][1]
print(f"Type: {type(msg2).__name__}")
print(f"Has tool_calls: {bool(msg2.tool_calls)}")
print(f"\nTool Calls Detail:")
print(msg2.tool_calls)
print("\n→ LLM analyzed query and decided to call 'currency_converter'.")
print("→ LLM extracted parameters from natural language.")
print("→ LLM did NOT execute the tool—just requested it.")

In [ ]:
# Message 3: ToolMessage (Tool Execution Result)
print("MESSAGE 3: ToolMessage")
print("=" * 70)
msg3 = result['messages'][2]
print(f"Type: {type(msg3).__name__}")
print(f"Tool Call ID: {msg3.tool_call_id}")
print(f"\nContent (Tool Output):")
print(msg3.content)
print("\n→ ToolNode executed: currency_converter.invoke({args})")
print("→ Result stored in ToolMessage for LLM to see.")

In [ ]:
# Message 4: AIMessage (Final Response)
print("MESSAGE 4: AIMessage (Final Response)")
print("=" * 70)
msg4 = result['messages'][3]
print(f"Type: {type(msg4).__name__}")
print(f"Has tool_calls: {bool(msg4.tool_calls) if hasattr(msg4, 'tool_calls') else False}")
print(f"\nContent:")
print(msg4.content)
print("\n→ LLM saw the tool result and formulated a natural language answer.")
print("→ No more tool_calls = Router sends to END.")

### Reference Point: The Two LLM Calls

Single tool execution involves **exactly 2 LLM calls**:

```
┌─────────────────────────────────────────────────────────────────────┐
│ LLM CALL 1: "What should I do with this query?"                    │
│   Input:  [HumanMessage("What is 1000 USD in EUR?")]               │
│   Output: AIMessage(tool_calls=[{currency_converter, args}])       │
│   Decision: "I need to call currency_converter"                    │
└─────────────────────────────────────────────────────────────────────┘
                              ↓
            [ToolNode executes currency_converter]
                              ↓
┌─────────────────────────────────────────────────────────────────────┐
│ LLM CALL 2: "Now I have the result. What should I tell the user?" │
│   Input:  [HumanMessage, AIMessage(tool_calls), ToolMessage]       │
│   Output: AIMessage(content="1000 USD equals 920 EUR...")          │
│   Decision: "I have enough info. Here's my answer."                │
└─────────────────────────────────────────────────────────────────────┘
```

---

## Section 6: Visualize the Conversation Flow

In [ ]:
print("CONVERSATION FLOW VISUALIZATION")
print("=" * 70)

for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER:")
        print(f"    \"{msg.content}\"")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT → Calling tool:")
            for tc in msg.tool_calls:
                print(f"    Tool: {tc['name']}")
                print(f"    Args: {tc['args']}")
        else:
            print(f"\n[{i}] 🤖 AGENT → Final response:")
            print(f"    \"{msg.content}\"")
            
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL RESULT:")
        # Indent each line of the tool output
        for line in msg.content.split('\n'):
            print(f"    {line}")

print("\n" + "=" * 70)

---

## Section 7: Test with EMI Calculator

Let's verify the same pattern with a different tool.

**Query:** "Calculate EMI for a 50000 USD loan at 7.5% for 36 months"

In [ ]:
# Create new state with EMI query
state2 = {
    "messages": [
        HumanMessage(content="Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
    ]
}

print("Query: Calculate EMI for a 50000 USD loan at 7.5% for 36 months")
print("=" * 70)

In [ ]:
# Execute
result2 = app.invoke(state2)

print(f"\n✅ Execution complete")
print(f"Message count: {len(result2['messages'])} (expected: 4)")

In [ ]:
# Verify the tool called
print("Tool Selection Verification:")
print("=" * 70)
tool_call_msg = result2['messages'][1]
if tool_call_msg.tool_calls:
    tc = tool_call_msg.tool_calls[0]
    print(f"✅ LLM selected: {tc['name']}")
    print(f"\nExtracted parameters:")
    for key, value in tc['args'].items():
        print(f"   {key}: {value}")

In [ ]:
# Display the conversation flow
print("\nCONVERSATION FLOW:")
print("=" * 70)

for i, msg in enumerate(result2["messages"], 1):
    msg_type = type(msg).__name__
    
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER: \"{msg.content[:50]}...\"")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT: Calling {msg.tool_calls[0]['name']}")
        else:
            preview = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
            print(f"\n[{i}] 🤖 AGENT: \"{preview}\"")
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL: Executed emi_calculator")

In [ ]:
# Show final response
print("FINAL RESPONSE:")
print("=" * 70)
print(result2["messages"][-1].content)

---

## Section 8: Key Observations

### Reference Point: Single Tool Execution Pattern

| Aspect | Observation |
|--------|-------------|
| **LLM Calls** | Exactly 2 (decide + respond) |
| **Messages** | Exactly 4 (Human → AI+tools → Tool → AI) |
| **Loop Iterations** | 1 (agent → tools → agent → END) |
| **Tool Selection** | LLM chooses based on docstring match |
| **Parameter Extraction** | LLM parses natural language to typed args |

### What the LLM Does Autonomously

✅ **Identifies correct tool** from docstring descriptions  
✅ **Extracts parameters** from natural language query  
✅ **Decides when to stop** (no more tools needed)  
✅ **Formulates response** using tool output  

### What You Define (Not the LLM)

✅ Tool functions with clear docstrings  
✅ Graph structure (nodes + edges)  
✅ Router logic (check for tool_calls)  

> **Key Insight:** No hardcoded "if currency question → call currency_converter" logic. The LLM orchestrates tool selection based on semantic understanding.

### Reference Point: State Evolution

```
START:       [HumanMessage]                          → 1 message
After agent: [HumanMessage, AIMessage(tool_calls)]   → 2 messages  
After tools: [HumanMessage, AIMessage, ToolMessage]  → 3 messages
After agent: [HumanMessage, AIMessage, ToolMessage, AIMessage] → 4 messages
END:         Final state preserved
```

**State only grows, never shrinks.** This preserves full conversation history for:
- Multi-turn conversations
- Debugging and tracing
- Context for future tool calls

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **Execution Flow** | START → agent → tools → agent → END |
| **LLM Calls** | 2 calls: (1) decide tool, (2) generate response |
| **Message Types** | Human → AI(tool_calls) → Tool → AI(content) |
| **Message Count** | Single tool = 4 messages always |
| **LLM Autonomy** | Selects tool, extracts params, decides when done |
| **State Growth** | Messages accumulate, never removed |

## Critical Understanding

```
llm_with_tools.invoke() → LLM decides WHAT to call (returns tool_calls)
ToolNode → Actually EXECUTES the tool (returns ToolMessage)
```

The LLM never executes tools directly—it only requests them!

## Next Steps

In **Notebook 07: Parallel Execution**, we'll explore what happens when:
- Query requires multiple independent tools
- LLM calls all tools in a single request
- ToolNode executes them simultaneously

---